# Install Dependencies

In [ ]:
# Install required packages
!pip install yt-dlp
!pip install soundfile
!pip install librosa
!pip install tensorflow
!pip install tensorflow-hub

print("All dependencies installed")

All dependencies installed


# Imports

In [ ]:
import os
import re
import csv
import io
import urllib.request
import random
import tempfile
import subprocess
import json
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from collections import defaultdict
from IPython.display import Audio, display

import soundfile as sf
import librosa
import tensorflow as tf
import tensorflow_hub as hub

# random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class Config:
    # audio processing
    TARGET_SR = 16000
    CLIP_SEC = 3.0
    HOP_SEC = 1.5
    RMS_MIN_DB = -45
    RMS_TOP_DB = 30

    # YAMNet
    YAMNET_THRESHOLD = 0.08
    SPEECH_BLOCK_THRESHOLD = 0.20
    DOG_LABEL_SUBSTRINGS = ["Dog", "Bark", "Growl", "Howl", "Whimper", "Whine", "Yip"]
    SPEECH_BLOCKLIST = [
        "Speech", "Narration", "Monologue", "Conversation",
        "Babbling", "Whispering", "Shout", "Laughter", "Music", "Singing"
    ]

    # paths
    COOKIES_PATH = "/content/www.youtube.com_cookies.txt"
    DATASET_ROOT = "/content/dog_disease_dataset"

config = Config()

# function for subprocess
def run(cmd, **kwargs):
    kwargs.setdefault("check", True)
    kwargs.setdefault("text", True)
    return subprocess.run(cmd, **kwargs)

print("Core imports and configuration complete")

Core imports and configuration complete


# Cookie Setup

In [ ]:
from google.colab import files

if not Path(config.COOKIES_PATH).exists():
    print("Please upload your YouTube cookies TXT file")
    uploaded = files.upload()
    name = next(iter(uploaded))
    if name != "www.youtube.com_cookies.txt":
        os.rename(name, "www.youtube.com_cookies.txt")
    config.COOKIES_PATH = "/content/www.youtube.com_cookies.txt"

assert Path(config.COOKIES_PATH).exists(), f"Cookies not found at {config.COOKIES_PATH}"
print("Using cookies:", config.COOKIES_PATH)

Using cookies: /content/www.youtube.com_cookies.txt


# Audio Processing Utilities

In [ ]:
def ffmpeg_to_wav16k(input_path, out_path):
    run(["ffmpeg", "-hide_banner", "-loglevel", "error",
         "-i", str(input_path), "-ac", "1", "-ar", str(config.TARGET_SR), "-y", str(out_path)])

def slice_audio(y, sr, length_sec, hop_sec):
    L = int(length_sec * sr)
    H = int(hop_sec * sr)
    for start in range(0, max(len(y) - L + 1, 0), H):
        yield y[start:start+L]

def rms_db(y):
    rms = np.sqrt(np.mean(y**2) + 1e-10)
    return 20 * np.log10(rms + 1e-10)

def trim_silence(y, sr, top_db=config.RMS_TOP_DB):
    yt, _ = librosa.effects.trim(y, top_db=top_db)
    return yt

print("Audio processing utilities defined")

Audio processing utilities defined


# YAMNet Setup and Utilities

In [ ]:
# load YAMNet
YAMNET = hub.load("https://tfhub.dev/google/yamnet/1")

# class names
with urllib.request.urlopen(
    "https://raw.githubusercontent.com/tensorflow/models/master/research/audioset/yamnet/yamnet_class_map.csv"
) as f:
    YAMNET_CLASSES = [row["display_name"] for row in csv.DictReader(io.StringIO(f.read().decode("utf-8")))]

print(f"YAMNet loaded with {len(YAMNET_CLASSES)} classes")

def enhanced_dog_detection_with_retry(y, sr, max_retries=2):
    for attempt in range(max_retries):
        try:
            if sr != config.TARGET_SR:
                y = librosa.resample(y, orig_sr=sr, target_sr=config.TARGET_SR)
                sr = config.TARGET_SR

            # ensure audio is not empty and long enough
            if len(y) < int(0.96 * sr):
                padding = int(0.96 * sr) - len(y)
                y = np.pad(y, (0, padding), mode='constant')

            waveform = y.astype(np.float32)
            scores, embeddings, spectrogram = YAMNET(waveform)
            mean_scores = tf.reduce_mean(scores, axis=0).numpy()

            cls2p = {YAMNET_CLASSES[i]: float(mean_scores[i]) for i in range(len(YAMNET_CLASSES))}

            # check for speech/music
            speech_music_score = max((cls2p.get(name, 0.0) for name in [
                "Speech", "Narration", "Conversation", "Music", "Singing"
            ]), default=0.0)

            if speech_music_score > 0.25:
                return False, {"reason": f"speech_music_too_high:{speech_music_score:.3f}"}

            # check for dog sounds
            dog_scores = {}
            for cls_name, score in cls2p.items():
                cls_lower = cls_name.lower()
                if any(dog_term in cls_lower for dog_term in ["dog", "bark", "howl", "whine", "growl"]):
                    dog_scores[cls_name] = score

            if dog_scores:
                max_dog_score = max(dog_scores.values())
                if max_dog_score > 0.05:
                    return True, {"dog_scores": dog_scores, "speech_music": speech_music_score}
                else:
                    return False, {"reason": f"dog_sounds_too_weak:{max_dog_score:.3f}"}
            else:
                return False, {"reason": "no_dog_sounds_detected"}

        except Exception as e:
            if attempt == max_retries - 1:
                return False, {"reason": f"error:{str(e)}"}
            continue

    return False, {"reason": "max_retries_exceeded"}

print("Enhanced dog detection")

YAMNet loaded with 521 classes
Enhanced dog detection


# Query Definitions

In [ ]:
EXPANDED_DISEASE_QUERIES = {
    "kennel_cough": [
        "dog kennel cough", "bordetella cough", "CIRDC dog",
        "canine infectious cough", "dog honking cough",
        "dog coughing fit", "dog coughing attack", "puppy coughing",
        "dog respiratory infection", "dog breathing problems",
        "perro tos perreras", "chien toux chenil", "hund zwingerhusten",
        "cachorro tossindo", "cane tosse"
    ],
    "tracheal_collapse": [
        "dog tracheal collapse", "goose honk cough", "small dog coughing",
        "toy breed breathing", "pomeranian cough", "yorkie coughing",
        "chihuahua breathing", "miniature dog cough",
        "perro colapso traqueal", "chien trachee effondree"
    ],
    "reverse_sneeze": [
        "dog reverse sneeze", "dog snorting attack", "dog pharyngeal gag",
        "dog inspiratory paroxysm", "dog breathing spasm",
        "perro estornudo inverso", "chien eternuement inverse"
    ]
}

NORMAL_SOUND_QUERIES = {
    "normal_barking": [
        "dog barking", "dog bark", "canine barking", "dog vocalization",
        "perro ladrando", "chien aboie", "hund bellen"
    ],
    "normal_whining": [
        "dog whining", "puppy crying", "dog whimpering",
        "dog complaining sounds", "perro llorando"
    ],
    "normal_howling": [
        "dog howling", "wolf howl dog", "dog singing",
        "husky howling", "perro aullando"
    ],
    "normal_breathing": [
        "dog breathing", "dog panting", "puppy breathing",
        "dog sleep sounds", "perro respirando"
    ],
    "normal_playing": [
        "dog play sounds", "puppy playing", "dog happy sounds",
        "dog excitement sounds"
    ]
}

print("Query definitions loaded")

Query definitions loaded


# Core Collection Functions

In [ ]:
def download_videos_for_query(query, tmp_dir, max_videos):
    try:
        cmd = [
            "yt-dlp", "--cookies", config.COOKIES_PATH,
            "-x", "--audio-format", "wav", "--audio-quality", "0",
            "--no-playlist", "--match-filter", "!is_live & duration < 240",
            "--ignore-errors", "--retries", "2",
            "--max-downloads", str(max_videos),
            "-o", f"{tmp_dir}/%(id)s.%(ext)s",
            f"ytsearch{max_videos}:{query}"
        ]

        result = run(cmd, capture_output=True, timeout=180, check=False)
        if result.returncode == 0:
            return [f.stem for f in Path(tmp_dir).glob("*.wav")]
    except Exception as e:
        print(f"Error downloading videos: {e}")
    return []

def process_video(video_id, tmp_dir, category, output_dir, max_clips=6):
    """Process a single video and extract valid dog sounds"""
    audio_file = Path(tmp_dir) / f"{video_id}.wav"
    if not audio_file.exists():
        return 0

    clips_saved = 0

    try:
        # convert to 16kHz
        wav_16k = Path(tmp_dir) / f"{video_id}_16k.wav"
        ffmpeg_to_wav16k(audio_file, wav_16k)

        # load and process
        y, sr = sf.read(wav_16k)
        if y.ndim > 1:
            y = y.mean(axis=1)

        # extract segments throughout the audio
        segments = list(slice_audio(y, sr, config.CLIP_SEC, config.CLIP_SEC * 0.6))
        segments = segments[:max_clips * 2]

        for seg_idx, segment in enumerate(segments):
            if clips_saved >= max_clips:
                break

            if len(segment) < config.TARGET_SR * 0.7:
                continue
            if rms_db(segment) < -42:
                continue

            # YAMNet verification
            is_dog_sound, confidence = enhanced_dog_detection_with_retry(segment, sr)
            if not is_dog_sound:
                continue

            # save verified dog sound
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            out_name = f"{video_id}_{timestamp}_{seg_idx:03d}.wav"
            out_path = output_dir / out_name

            sf.write(out_path, segment, config.TARGET_SR, subtype="PCM_16")
            clips_saved += 1

    except Exception as e:
        print(f"Error processing {video_id}: {e}")

    return clips_saved

def collect_category(category, queries, output_root, target_count, max_videos=10,
                    max_clips_per_video=6, skip_existing=False):
    category_dir = output_root / category
    category_dir.mkdir(exist_ok=True)

    if skip_existing:
        existing_files = list(category_dir.glob("*.wav"))
        existing_videos = set(f.name.split('_')[0] for f in existing_files)
    else:
        existing_videos = set()

    current_count = len(existing_files) if skip_existing else 0
    collected_videos = set(existing_videos)

    for query in queries:
        if current_count >= target_count:
            break

        print(f"Query: '{query}'")

        with tempfile.TemporaryDirectory() as tmp_dir:
            video_ids = download_videos_for_query(query, tmp_dir, max_videos)

            for video_id in video_ids:
                if current_count >= target_count:
                    break
                if video_id in collected_videos:
                    continue

                collected_videos.add(video_id)
                clips_saved = process_video(
                    video_id=video_id,
                    tmp_dir=tmp_dir,
                    category=category,
                    output_dir=category_dir,
                    max_clips=max_clips_per_video
                )

                current_count += clips_saved
                print(f" Video {video_id}: {clips_saved} clips (Total: {current_count}/{target_count})")

    return current_count

# Data Augmentation Function

In [ ]:
def augment_audio_clips(audio_files, target_count, augmentation_factor=3):
    import librosa.effects as effects

    augmented_files = list(audio_files)
    needed = target_count - len(audio_files)

    if needed <= 0:
        return audio_files

    print(f"Augmenting {len(audio_files)} clips to reach {target_count}...")

    for i in range(needed):
        try:
            # select a random source file
            src_file = random.choice(audio_files)
            y, sr = sf.read(src_file)

            # apply multiple augmentations
            aug_y = y.copy()

            # pitch shift
            if random.random() > 0.5:
                aug_y = effects.pitch_shift(aug_y, sr=sr, n_stones=random.uniform(-2, 2))

            # time stretch
            if random.random() > 0.5:
                aug_y = effects.time_stretch(aug_y, rate=random.uniform(0.8, 1.2))

            # volume change
            aug_y = aug_y * random.uniform(0.7, 1.3)

            # background noise
            if random.random() > 0.7:
                noise = np.random.normal(0, 0.005, len(aug_y))
                aug_y = aug_y + noise

            # create augmented filename
            aug_path = src_file.parent / f"aug_{i}_{src_file.name}"
            sf.write(aug_path, aug_y, sr, subtype="PCM_16")
            augmented_files.append(aug_path)

        except Exception as e:
            print(f"Augmentation error: {e}")
            continue

    return augmented_files

# Enhanced Collection with Retries

In [ ]:
def collect_with_retries(category, queries, target_count, output_dir, max_passes=3):
    collected_files = []
    pass_count = 0

    while len(collected_files) < target_count and pass_count < max_passes:
        pass_count += 1
        print(f"Pass {pass_count} for {category}...")

        current_count = collect_category(
            category=category,
            queries=queries,
            output_root=output_dir,
            target_count=target_count - len(collected_files),
            max_videos=15,
            max_clips_per_video=10,
            skip_existing=(pass_count > 1)
        )

        # get actual file paths
        category_dir = output_dir / category
        current_files = list(category_dir.glob("*.wav"))
        collected_files = current_files

        print(f"After pass {pass_count}: {len(collected_files)}/{target_count}")

    return collected_files

# Main Expansion Workflow

In [ ]:
def build_expanded_dataset(target_per_class=150):
    output_root = Path("/content/expanded_dog_dataset")
    output_root.mkdir(exist_ok=True)

    print(f"Target: {target_per_class} clips per category")
    print("=" * 60)

    # collect disease sounds with expanded queries
    print("\nPHASE 1: Collecting Disease Sounds")
    print("-" * 40)

    disease_counts = {}
    for category, queries in EXPANDED_DISEASE_QUERIES.items():
        print(f"\nCollecting: {category}")
        files = collect_with_retries(category, queries, target_per_class, output_root)
        disease_counts[category] = len(files)
        print(f"{category}: {len(files)} clips")

        # augment if needed
        if len(files) < target_per_class:
            augmented_files = augment_audio_clips(files, target_per_class)
            disease_counts[category] = len(augmented_files)
            print(f"Augmented {category}: {len(augmented_files)} clips")

    # Collect normal sounds
    print("\nPHASE 2: Collecting Normal Sounds")
    print("-" * 40)

    normal_counts = {}
    for category, queries in NORMAL_SOUND_QUERIES.items():
        print(f"\nCollecting: {category}")
        files = collect_with_retries(category, queries, target_per_class, output_root)
        normal_counts[category] = len(files)
        print(f"{category}: {len(files)} clips")

    # final summary
    print("\n" + "=" * 60)
    print("FINAL EXPANDED DATASET SUMMARY")
    print("=" * 60)

    total = 0
    all_counts = {**disease_counts, **normal_counts}
    for category, count in all_counts.items():
        print(f"{category:25s}: {count:3d} clips")
        total += count

    print(f"{'TOTAL':25s}: {total:3d} clips")
    print(f"{'NUMBER OF CLASSES':25s}: {len(all_counts)}")

    return output_root

# run
print("Starting expanded dataset collection.")
expanded_dataset = build_expanded_dataset(target_per_class=150)

Starting expanded dataset collection.
Target: 150 clips per category

PHASE 1: Collecting Disease Sounds
----------------------------------------

Collecting: kennel_cough
Pass 1 for kennel_cough...
Query: 'dog kennel cough'
 Video K387RyCS5Lc: 0 clips (Total: 0/150)
 Video 57mCNtlDDjM: 0 clips (Total: 0/150)
 Video SKjOEe2qzd0: 0 clips (Total: 0/150)
 Video -DD2Ic_vwx4: 0 clips (Total: 0/150)
 Video jadWV_azIiQ: 0 clips (Total: 0/150)
 Video HbCXGL1gPLQ: 0 clips (Total: 0/150)
 Video GBEDWjqG1qU: 4 clips (Total: 4/150)
 Video bcQ5K2nM2Kg: 0 clips (Total: 4/150)
Query: 'bordetella cough'
 Video 31tnXPlhA7w: 3 clips (Total: 7/150)
 Video S3oZrMGDMMw: 0 clips (Total: 7/150)
 Video Kw7vfuN0G3I: 0 clips (Total: 7/150)
 Video Aoz6omYXy8o: 0 clips (Total: 7/150)
 Video NydZooWrVy0: 0 clips (Total: 7/150)
 Video tUKkpNXM-Qc: 0 clips (Total: 7/150)
 Video XK6ij4pFavo: 0 clips (Total: 7/150)
Query: 'CIRDC dog'
 Video UYwzpLQVzm0: 0 clips (Total: 7/150)
 Video g2I2ptRsoUM: 0 clips (Total: 7/150)

# Save and Download Dataset

In [12]:
import zipfile
from google.colab import files

# zip file
zip_path = "/content/dog_disease_dataset.zip"
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for audio_file in Path("/content/expanded_dog_dataset").rglob("*.wav"):
        zipf.write(audio_file)

print("Dataset zipped")
files.download(zip_path)
print("Download started")

Dataset zipped


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download started
